In [ ]:
PATIENTID = '495425e0-b16b-0baa-f926-0dfbe9870673'
PATIENTID = PATIENTID.upper()
print(PATIENTID)

In [ ]:
neo4j_url = "bolt://localhost:7687" 
neo4j_user = "neo4j" 
neo4j_password = "***"
neo4j_db = "poc"
SOURCE_DATASET_LOCATION = "/Users/amk/#Datasets/synthea/csv/synthea50"

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
        .master('spark://Adityas-MacBook-Pro.local:7077')
        .appName('HealthConnect Test')
        .config('spark.ui.port', '4051')
        .config('spark.sql.repl.eagerEval.enabled', True)
        .config('spark.jars', '/Users/amk/Github/healthconnect/damg7374/code/utilities/neo4j-spark-connector.jar')
        .config("neo4j.url", neo4j_url)
        .config("neo4j.authentication.type", "basic")
        .config("neo4j.authentication.basic.username", neo4j_user)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.authentication.basic.password", neo4j_password)
        .config("neo4j.database", neo4j_db)
        .config('spark.executor.instances', 1)
        .config('spark.executor.cores', 1) 
        .config('spark.executor.memory', '1G') 
        .config('spark.cores.max', 1) 
        .getOrCreate())


In [ ]:
print("================")
print("patients.csv")
print("================")
SOURCE_DATASET_FILENAME = "patients.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(Id) = "{PATIENTID}"')
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")


In [ ]:
print("================")
print("encounters.csv")
print("================")

SOURCE_DATASET_FILENAME = "encounters.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(PATIENT) = "{PATIENTID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

a = df.filter(f'upper(PATIENT) = "{PATIENTID}"').orderBy('Id').selectExpr('Id as ENCOUNTER','ORGANIZATION', 'PROVIDER', 'PAYER')
b = a.toPandas().to_dict(orient='list')
# b

In [ ]:
print("================")
print("organizations.csv")
print("================")

from pyspark.sql.functions import col
c = b['ORGANIZATION']
SOURCE_DATASET_FILENAME = "organizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("providers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PROVIDER']
SOURCE_DATASET_FILENAME = "providers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("payers.csv")
print("================")

from pyspark.sql.functions import col
c = b['PAYER']
SOURCE_DATASET_FILENAME = "payers.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(col("Id").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("claims.csv")
print("================")

SOURCE_DATASET_FILENAME = "claims.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(PATIENTID) = "{PATIENTID}"').orderBy('Id')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

x = df.filter(f'upper(PATIENTID) = "{PATIENTID}"').orderBy('Id').selectExpr('Id as CLAIM')
y = x.toPandas().to_dict(orient='list')
b.update(y)
# b

In [ ]:
print("================")
print("claims_transactions.csv")
print("================")

SOURCE_DATASET_FILENAME = "claims_transactions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
a = df.filter(f'upper(PATIENTID) = "{PATIENTID}"').orderBy('CLAIMID', 'CHARGEID')

a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

x = df.filter(f'upper(PATIENTID) = "{PATIENTID}"').orderBy('Id').selectExpr('Id as CLAIM_TRANSACTION')
y = x.toPandas().to_dict(orient='list')
b.update(y)
# b


In [ ]:
print("================")
print("allergies.csv")
print("================")
SOURCE_DATASET_FILENAME = "allergies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("careplans.csv")
print("================")
SOURCE_DATASET_FILENAME = "careplans.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("conditions.csv")
print("================")
SOURCE_DATASET_FILENAME = "conditions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("devices.csv")
print("================")

SOURCE_DATASET_FILENAME = "devices.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("imaging_studies.csv")
print("================")

SOURCE_DATASET_FILENAME = "imaging_studies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("immunizations.csv")
print("================")

SOURCE_DATASET_FILENAME = "immunizations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("medications.csv")
print("================")
SOURCE_DATASET_FILENAME = "medications.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("observations.csv")
print("================")
SOURCE_DATASET_FILENAME = "observations.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("procedures.csv")
print("================")
SOURCE_DATASET_FILENAME = "procedures.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("supplies.csv")
print("================")
SOURCE_DATASET_FILENAME = "supplies.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = b['ENCOUNTER']
a = df.filter(col("ENCOUNTER").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
print("================")
print("payer_transitions.csv")
print("================")
SOURCE_DATASET_FILENAME = "payer_transitions.csv"
df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
c = [PATIENTID.lower()]
a = df.filter(col("PATIENT").isin(c))
a.show(a.count(), truncate=False)
print(f"Record count : {a.count()}")

In [ ]:
b

In [ ]:
a = ['allergies.csv', 'conditions.csv', 'observations.csv', 'procedures.csv', 'medications.csv', 'immunizations.csv', 'devices.csv', 'supplies.csv', 'careplans.csv', 'imaging_studies.csv']

SOURCE_DATASET_LOCATION = '/Users/amk/#Datasets/synthea/csv/synthea50'
for i in a:
    SOURCE_DATASET_FILENAME = i
    df = (spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{SOURCE_DATASET_FILENAME}", header=True, inferSchema=True))
    df.select('ENCOUNTER').distinct()

In [ ]:
from pyspark.sql import SparkSession
from functools import reduce

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Find Common ENCOUNTERs") \
    .getOrCreate()

# List of CSV files
files = [
    'allergies.csv', 'conditions.csv', 'observations.csv', 'procedures.csv',
    'medications.csv', 'immunizations.csv', 'devices.csv', 'supplies.csv',
    'careplans.csv', 'imaging_studies.csv'
]

SOURCE_DATASET_LOCATION = '/Users/amk/#Datasets/synthea/csv/synthea50'

# Read all files and select only the ENCOUNTER column
dfs = [spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{file}", header=True, inferSchema=True).select("ENCOUNTER").distinct() for file in files]

# Find common ENCOUNTER values across all files using an inner join
common_encounters = reduce(lambda df1, df2: df1.join(df2, on="ENCOUNTER", how="inner"), dfs)

# Show the common ENCOUNTER values
common_encounters.show()


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, count

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Find Most Frequent ENCOUNTER") \
    .getOrCreate()

# List of CSV files
files = [
    'allergies.csv', 'conditions.csv', 'observations.csv', 'procedures.csv',
    'medications.csv', 'immunizations.csv', 'devices.csv', 'supplies.csv',
    'careplans.csv', 'imaging_studies.csv'
]

# Add a file identifier and union all dataframes
dfs_with_file_ids = [
    spark.read.csv(f"{SOURCE_DATASET_LOCATION}/{file}", header=True)
    .select("ENCOUNTER")
    .distinct()
    .withColumn("file", lit(file)) for file in files
]

# Union all DataFrames
union_df = reduce(lambda df1, df2: df1.union(df2), dfs_with_file_ids)

# Count occurrences of each ENCOUNTER across files
encounter_counts = union_df.groupBy("ENCOUNTER").agg(count("file").alias("file_count"))

# Find the ENCOUNTER with the maximum count
most_common_encounter = encounter_counts.orderBy(col("file_count").desc())

# Show the result
most_common_encounter.show(truncate=False)
